In [2]:
import pandas as pd 
import numpy as np 
import seaborn as sns 
import matplotlib.pyplot as plt
import geoplotlib
import json
import folium
import os
from folium import plugins
from folium.plugins import *
from folium.utilities import *
from folium.features import *
import requests
from xml.etree import ElementTree
import branca.colormap as cm

In [3]:
#Importing Datasets

data = pd.read_csv('Data/5d067e6d8518921094fdab22_Accidents.csv')


In [4]:
data.columns

Index(['Unnamed: 0', 'Accident_Time', 'Accident_Date_G', 'Rno', 'Road_Type',
       'Region', 'Road_Status', 'Weather_Status', 'Dead_Number',
       'Injuries_Number', 'Damage_Road_Type', 'Accident_Type', 'Car',
       'Small_Truck', 'Big_Truck', 'Other_Car', 'Driver', 'Way', 'Vehicle',
       'Animal', 'Vehicles_Number', 'Paints', 'Eyes', 'Road_Close_Days',
       'Road_Close_Hours', 'GEOMETRIC_RoadTYPE', 'long', 'lat'],
      dtype='object')

In [8]:
#importing Json file 
Saudi_geo = os.path.join('Data', 'regions.json')
geo_json_data = json.load(open(Saudi_geo))

#putting airport locations in a list
locations1 = data[data.Dead_Number == 0][['lat', 'long']].dropna()
locationlist1 = locations1.values.tolist()
len(locationlist1)

locations2 = data[data.Dead_Number >= 1][['lat', 'long']].dropna()
locationlist2 = locations2.values.tolist()

# Airport_names = airports['Name'].apply(str)
# nameslist = Airport_names.values.tolist()
# len(nameslist)


Regions = data[['Region' ]].drop_duplicates()
Regions['Region'] = Regions['Region'].astype('str') 
Regions['Region'] = Regions['Region'].str.strip()
Regions.head()
Regions.dtypes


Region    object
dtype: object

In [19]:
Regions.head()

,Region
0,Qassim
4,Hail
8,Madina
41,EP
51,Riyadh


In [11]:
#Creating the map 
m= folium.Map(location=[data['lat'].mean(),data['long'].mean()],
              zoom_start=3,
              tiles = 'Mapbox Bright')

m.add_tile_layer(tiles='stamenterrain', name = 'stamenterrain')

#creating a Choropleth of GDP on the map  
# folium.Choropleth(geo_data=Saudi_geo,
#                   name=' GDP Choropleth',
#                   data=countries,
#                   columns=['ID','GDP'],
#                   key_on='feature.id',
#                   fill_color='YlGn',
#                   fill_opacity=0.7,
#                   line_opacity=0.5,
#                   legend_name='GDP ($ per capita)',
#                   reset=True).add_to(m)


#Adding Airports points to the map
airports_markers1 = folium.FeatureGroup(name=" Accedents no death ")
for point in range(0, len(locationlist1)):
    airports_markers1.add_child(folium.CircleMarker(location=locationlist1[point],radius=1))

airports_markers2 = folium.FeatureGroup(name=" Accedents death ")
for point in range(0, len(locationlist2)):
    airports_markers2.add_child(folium.CircleMarker(location=locationlist2[point],radius=1, color='Red'))
       

#adding the layer controller   
m.add_child(airports_markers1)
m.add_child(airports_markers2)
m.add_child(folium.LayerControl())

m.save('accidents.html')